In [1]:
from __future__ import absolute_import, division, print_function

from copy import deepcopy
import json
import glob
import os
import time
import argparse
import matplotlib.pyplot as plt
import numpy as np
# import seaborn as sns
#
# sns.set_style('white')
# sns.set_context('poster')
from pygments.lexer import default



COLORS = [
    '#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c',
    '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5',
    '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f',
    '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']


def calc_iou_individual(pred_box, gt_box):
    """Calculate IoU of single predicted and ground truth box

    Args:
        pred_box (list of floats): location of predicted object as
            [xmin, ymin, xmax, ymax]
        gt_box (list of floats): location of ground truth object as
            [xmin, ymin, xmax, ymax]

    Returns:
        float: value of the IoU for the two boxes.

    Raises:
        AssertionError: if the box is obviously malformed
    """
    x1_t, y1_t, x2_t, y2_t = gt_box
    x1_p, y1_p, x2_p, y2_p = pred_box

    if (x1_p > x2_p) or (y1_p > y2_p):
        raise AssertionError(
            "Prediction box is malformed? pred box: {}".format(pred_box))
    if (x1_t > x2_t) or (y1_t > y2_t):
        raise AssertionError(
            "Ground Truth box is malformed? true box: {}".format(gt_box))

    if (x2_t < x1_p or x2_p < x1_t or y2_t < y1_p or y2_p < y1_t):
        return 0.0

    far_x = np.min([x2_t, x2_p])
    near_x = np.max([x1_t, x1_p])
    far_y = np.min([y2_t, y2_p])
    near_y = np.max([y1_t, y1_p])

    inter_area = (far_x - near_x + 1) * (far_y - near_y + 1)
    true_box_area = (x2_t - x1_t + 1) * (y2_t - y1_t + 1)
    pred_box_area = (x2_p - x1_p + 1) * (y2_p - y1_p + 1)
    iou = inter_area / (true_box_area + pred_box_area - inter_area)
    return iou


def get_single_image_results(gt_boxes, pred_boxes, iou_thr):
    """Calculates number of true_pos, false_pos, false_neg from single batch of boxes.

    Args:
        gt_boxes (list of list of floats): list of locations of ground truth
            objects as [xmin, ymin, xmax, ymax]
        pred_boxes (dict): dict of dicts of 'boxes' (formatted like `gt_boxes`)
            and 'scores'
        iou_thr (float): value of IoU to consider as threshold for a
            true prediction.

    Returns:
        dict: true positives (int), false positives (int), false negatives (int)
    """

    all_pred_indices = range(len(pred_boxes))
    all_gt_indices = range(len(gt_boxes))
    if len(all_pred_indices) == 0:
        tp = 0
        fp = 0
        fn = len(gt_boxes)
        return {'true_pos': tp, 'false_pos': fp, 'false_neg': fn}
    if len(all_gt_indices) == 0:
        tp = 0
        fp = len(pred_boxes)
        fn = 0
        return {'true_pos': tp, 'false_pos': fp, 'false_neg': fn}

    gt_idx_thr = []
    pred_idx_thr = []
    ious = []
    for ipb, pred_box in enumerate(pred_boxes):
        for igb, gt_box in enumerate(gt_boxes):
            iou = calc_iou_individual(pred_box, gt_box)
            if iou > iou_thr:
                gt_idx_thr.append(igb)
                pred_idx_thr.append(ipb)
                ious.append(iou)

    args_desc = np.argsort(ious)[::-1]
    if len(args_desc) == 0:
        # No matches
        tp = 0
        fp = len(pred_boxes)
        fn = len(gt_boxes)
    else:
        gt_match_idx = []
        pred_match_idx = []
        for idx in args_desc:
            gt_idx = gt_idx_thr[idx]
            pr_idx = pred_idx_thr[idx]
            # If the boxes are unmatched, add them to matches
            if (gt_idx not in gt_match_idx) and (pr_idx not in pred_match_idx):
                gt_match_idx.append(gt_idx)
                pred_match_idx.append(pr_idx)
        tp = len(gt_match_idx)
        fp = len(pred_boxes) - len(pred_match_idx)
        fn = len(gt_boxes) - len(gt_match_idx)

    return {'true_pos': tp, 'false_pos': fp, 'false_neg': fn}


def calc_precision_recall(img_results):
    """Calculates precision and recall from the set of images

    Args:
        img_results (dict): dictionary formatted like:
            {
                'img_id1': {'true_pos': int, 'false_pos': int, 'false_neg': int},
                'img_id2': ...
                ...
            }

    Returns:
        tuple: of floats of (precision, recall)
    """
    true_pos = 0; false_pos = 0; false_neg = 0
    for _, res in img_results.items():
        true_pos += res['true_pos']
        false_pos += res['false_pos']
        false_neg += res['false_neg']

    try:
        precision = true_pos/(true_pos + false_pos)
    except ZeroDivisionError:
        precision = 0.0
    try:
        recall = true_pos/(true_pos + false_neg)
    except ZeroDivisionError:
        recall = 0.0

    return (precision, recall)


def get_model_scores_map(pred_boxes):
    """Creates a dictionary of from model_scores to image ids.

    Args:
        pred_boxes (dict): dict of dicts of 'boxes' and 'scores'

    Returns:
        dict: keys are model_scores and values are image ids (usually filenames)

    """
    model_scores_map = {}
    for img_id, val in pred_boxes.items():
        for score in val['scores']:
            if score not in model_scores_map.keys():
                model_scores_map[score] = [img_id]
            else:
                model_scores_map[score].append(img_id)
    return model_scores_map

def get_avg_precision_at_iou(gt_boxes, pred_boxes, iou_thr=0.5, scores_all_same=False):
    """Calculates average precision at given IoU threshold.

    Args:
        gt_boxes (list of list of floats): list of locations of ground truth
            objects as [xmin, ymin, xmax, ymax]
        pred_boxes (list of list of floats): list of locations of predicted
            objects as [xmin, ymin, xmax, ymax]
        iou_thr (float): value of IoU to consider as threshold for a
            true prediction.

    Returns:
        dict: avg precision as well as summary info about the PR curve

        Keys:
            'avg_prec' (float): average precision for this IoU threshold
            'precisions' (list of floats): precision value for the given
                model_threshold
            'recall' (list of floats): recall value for given
                model_threshold
            'models_thrs' (list of floats): model threshold value that
                precision and recall were computed for.
    """
    model_scores_map = get_model_scores_map(pred_boxes)
    sorted_model_scores = sorted(model_scores_map.keys())

    # Sort the predicted boxes in descending order (lowest scoring boxes first):
    for img_id in pred_boxes.keys():
        arg_sort = np.argsort(pred_boxes[img_id]['scores'])
        pred_boxes[img_id]['scores'] = np.array(pred_boxes[img_id]['scores'])[arg_sort].tolist()
        pred_boxes[img_id]['boxes'] = np.array(pred_boxes[img_id]['boxes'])[arg_sort].tolist()

    pred_boxes_pruned = deepcopy(pred_boxes)

    precisions = []
    recalls = []
    model_thrs = []
    img_results = {}

    if not scores_all_same:
        sorted_models_scores_list = sorted_model_scores[:-1]
    else:
        sorted_models_scores_list = sorted_model_scores

    # Loop over model score thresholds and calculate precision, recall
    for ithr, model_score_thr in enumerate(sorted_models_scores_list):
        # On first iteration, define img_results for the first time:
        img_ids = gt_boxes.keys() if ithr == 0 else model_scores_map[model_score_thr]
        for img_id in img_ids:
            gt_boxes_img = gt_boxes[img_id]
            box_scores = pred_boxes_pruned[img_id]['scores']
            start_idx = 0
            for score in box_scores:
                if score <= model_score_thr:
                    pred_boxes_pruned[img_id]
                    start_idx += 1
                else:
                    break

            if not scores_all_same:
                # Remove boxes, scores of lower than threshold scores:
                pred_boxes_pruned[img_id]['scores'] = pred_boxes_pruned[img_id]['scores'][start_idx:]
                pred_boxes_pruned[img_id]['boxes'] = pred_boxes_pruned[img_id]['boxes'][start_idx:]

            # Recalculate image results for this image
            img_results[img_id] = get_single_image_results(
                gt_boxes_img, pred_boxes_pruned[img_id]['boxes'], iou_thr)

        prec, rec = calc_precision_recall(img_results)
        precisions.append(prec)
        recalls.append(rec)
        model_thrs.append(model_score_thr)

    precisions = np.array(precisions)
    recalls = np.array(recalls)
    prec_at_rec = []
    for recall_level in np.linspace(0.0, 1.0, 11):
        try:
            args = np.argwhere(recalls >= recall_level).flatten()
            prec = max(precisions[args])
        except ValueError:
            prec = 0.0
        prec_at_rec.append(prec)
    avg_prec = np.mean(prec_at_rec)

    return {
        'avg_prec': avg_prec,
        'precisions': precisions,
        'recalls': recalls,
        'model_thrs': model_thrs}


def simple_precision_recall(gt_boxes, pred_boxes, iou_thr=0.5):
    img_results = {}
    img_ids = gt_boxes.keys()

    for img_id in img_ids:
        gt_boxes_img = gt_boxes[img_id]
        box_scores = pred_boxes[img_id]['scores']

        # Recalculate image results for this image
        img_results[img_id] = get_single_image_results(
            gt_boxes_img, pred_boxes[img_id]['boxes'], iou_thr)

    precision, recall = calc_precision_recall(img_results)
    return precision, recall

In [2]:
ground_truth_boxes = 'ground_truth_boxes.json'
predicted_boxes = 'predicted_boxes.json'

with open(ground_truth_boxes) as infile:
    gt_boxes = json.load(infile)

with open(predicted_boxes) as infile:
    pred_boxes = json.load(infile)

In [3]:
scales = {'small': [0, np.power(32, 2)],
         'medium': [np.power(32, 2), np.power(96, 2)],
         'large': [np.power(96, 2), np.infty]}

In [4]:
scales

{'large': [9216, inf], 'medium': [1024, 9216], 'small': [0, 1024]}

In [5]:
def box_area(box):
    x1, y1, x2, y2 = box
    
    if (x1 > x2) or (y1 > y2):
        raise AssertionError(
            "Prediction box is malformed? pred box: {}".format(pred_box))
        
    box_area = (x2 - x1 + 1) * (y2 - y1 + 1)
    
    return box_area
    

In [6]:
def filter_boxes(boxes, scale='all'):
    if scale != 'all':
        filtered_boxes = []
        for box in boxes:
            area = box_area(box)
            if area >= scales[scale][0] and area < scales[scale][1]:
                filtered_boxes.append(box)
            
        return filtered_boxes
    else:
        return boxes

In [7]:
def box_in_scale(box, scale='all'):
    if scale != 'all':   
        area = box_area(box)
        print(area)
        if area >= scales[scale][0] and area < scales[scale][1]:  
            return True
        else:
            return False
    else:
        return True

In [12]:
box_in_scale([360, 458, 385, 484], scale='small')

702


True

In [20]:
scales

{'large': [9216, inf], 'medium': [1024, 9216], 'small': [0, 1024]}

In [10]:
gt_boxes['img_00103.png']

[[847, 1003, 942, 1166], [546, 887, 656, 1147]]

In [23]:
test_ = filter_boxes(gt_boxes['img_00103.png'], scale='medium')

In [24]:
test_

[]

In [37]:


scale = 'small'
iou_thr = 0.5
img_results = {}
img_ids = gt_boxes.keys()

for img_id in img_ids:
    gt_boxes_img = gt_boxes[img_id] 
    filtered_gt_boxes = filter_boxes(gt_boxes_img, scale=scale)
    filtered_pred_boxes= filter_boxes(pred_boxes[img_id]['boxes'], scale=scale)
    # Recalculate image results for this image
    img_results[img_id] = get_single_image_results(
        filtered_gt_boxes, filtered_pred_boxes, iou_thr)

precision, recall = calc_precision_recall(img_results)
#f1_score = 

In [38]:
img_results

{'img_00103.png': {'false_neg': 0, 'false_pos': 1, 'true_pos': 0},
 'img_00105.png': {'false_neg': 0, 'false_pos': 2, 'true_pos': 0},
 'img_00106.png': {'false_neg': 0, 'false_pos': 1, 'true_pos': 0},
 'img_00107.png': {'false_neg': 0, 'false_pos': 1, 'true_pos': 0},
 'img_00108.png': {'false_neg': 0, 'false_pos': 1, 'true_pos': 0},
 'img_00112.png': {'false_neg': 0, 'false_pos': 1, 'true_pos': 0},
 'img_00113.png': {'false_neg': 0, 'false_pos': 1, 'true_pos': 0},
 'img_00114.png': {'false_neg': 0, 'false_pos': 1, 'true_pos': 0},
 'img_00115.png': {'false_neg': 0, 'false_pos': 1, 'true_pos': 0},
 'img_00116.png': {'false_neg': 0, 'false_pos': 1, 'true_pos': 0},
 'img_00117.png': {'false_neg': 0, 'false_pos': 1, 'true_pos': 0},
 'img_00118.png': {'false_neg': 0, 'false_pos': 1, 'true_pos': 0},
 'img_00119.png': {'false_neg': 0, 'false_pos': 1, 'true_pos': 0},
 'img_00120.png': {'false_neg': 0, 'false_pos': 1, 'true_pos': 0},
 'img_00121.png': {'false_neg': 0, 'false_pos': 1, 'true_pos':

In [39]:
precision

0.0

In [40]:
recall

0.0